
<img src="Images/ebay_logo.png">
# <center> Collecting eBay Coins Listings Data


<center> This script creates a CSV file of ebay listings with 
     
<center> 20 columns of data to use for analysis.
<br>

## <center> Important: 

<center> 1. This script is made to search completed ebay listings.
    
<center>2. Filter listings on eBay search before collect your input URL.
<center> 3. Go to page 2 of results to collect the url that ends with "_pgn=2"    

In [1]:
# How many results do you want?

desired_number_of_results = 150


# URL for page 1 of listings you want to scrape?

input_link = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=ms67+&_sacat=11633&LH_TitleDesc=0&_fsrp=1&_sop=1&LH_Complete=1&rt=nc&LH_TitleDesc=0&_pgn=2'


# Where Would you like to Save your CSV File?

csv_title = 'Data/Sample'


# File will be to saved to the location of this notebook.

## <center> Example of page ebay results page.
<img src="Images/ebay_search.png">
    

 <center> Don't forget to run input cell.
    

# <center> Time To Start Scraping
 <center> Now run cell below to get results!
    

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')



# Prepare inputs

partial_link = input_link[:-1]

d = desired_number_of_results/50
ru = math.ceil(d)

csv_title = f'{csv_title}.csv'

# Loop through ebay listings.

n = 0
ebay_listings = []

for r in range(ru):
    
    n = n+1
    url = f'{partial_link}{str(n)}' 
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('li', class_='s-item')
    
    
    for result in results: 
     
                title = result.find('h3', class_="s-item__title")
                title1 = list(title.strings)[-1]            
                l_title = len(title1)            
                status = list(title.strings)[0]
                s_split= status.split(" ")
                sold= s_split[0]
                price = result.find('div', class_="s-item__details").span.text    
                end_time = result.find('span', class_="s-item__ended-date").text
                time = end_time[7:]
                date = end_time[:6]
                year = status[-4:]
                link = result.a['href']
                
                try:
                    result.find('span', class_="s-item__shipping").text
                except AttributeError:
                     Shipping = 'Free Shipping'     
                else: 
                    shipping = result.find('span', class_="s-item__shipping").text
                
                try:
                    result.find('span', class_="s-item__bids").text                    
                except AttributeError:                   
                    bids= 'BIN'
                    f_type= 'Buy_it_now'
                else:
                    bids = result.find('span', class_="s-item__bids").text 
                    f_type = 'Auction'
                try:
                    result.find('span', class_="x-tooltip").text
                except AttributeError:
                    trp = 'No'
                else:
                    trp ='Yes'                
                try:
                    result.find('span', class_='s-item__free-returns').text
                except AttributeError:
                    returns = 'No'
                else:
                    returns= 'Yes'
                try:
                    result.find('span', class_='s-item__purchase-options s-item__purchaseOptions').text
                except AttributeError:
                    boa= 'No'
                else:
                    boa= 'Yes'

            
               
                
                #Add to list of listings
                ebay_listings.append({'Format': f_type,
                                      'Best_Offer_Accepted': boa,
                                      'Result': sold,                                   
                                      'Title': title1,
                                      'Price': price,
                                      'Bids': bids,
                                      'Shipping': shipping,
                                      'Free_Returns': returns,
                                      'Top_Rated_Plus': trp,
                                      'Sale_Date': date,
                                      'Sale_Time': time,
                                      'Sale_Year': year,
                                      'Link': link, 
                                      'Title_Length': l_title 
                                })
               
                
df = pd.DataFrame(ebay_listings)
df = df.iloc[:desired_number_of_results]

ebay_listings_df = df[['Result','Title','Format','Best_Offer_Accepted','Price','Bids','Shipping','Free_Returns','Top_Rated_Plus','Sale_Time','Sale_Date','Sale_Year','Link','Title_Length']]
# ebay_listings_df.to_csv(csv_title)

ebay_listings_df.head()

,Result,Title,Format,Best_Offer_Accepted,Price,Bids,Shipping,Free_Returns,Top_Rated_Plus,Sale_Time,Sale_Date,Sale_Year,Link,Title_Length
0,Sold,1954-S PCGS MS67RD 1C Lincoln Wheat Cent Penny...,Buy_it_now,No,$120.00,BIN,+$2.95 shipping,Yes,Yes,9:41,Mar-2,2019,https://www.ebay.com/itm/1954-S-PCGS-MS67RD-1C...,76
1,Sold,PCGS MS67 RD 2005 SATIN FINISH RED Lincoln Mem...,Buy_it_now,No,$19.99,BIN,+$5.53 shipping,No,No,8:43,Mar-2,2019,https://www.ebay.com/itm/PCGS-MS67-RD-2005-SAT...,80
2,Ended,"LOW RESERVE! Lot of 25 PCGS MS66RD, MS66+RD, ...",Auction,No,$639.50,0 bids,+$27.50 shipping,No,No,8:40,Mar-2,2019,https://www.ebay.com/itm/LOW-RESERVE-Lot-of-25...,79
3,Sold,2012 D PCGS MS67RD Red Lincoln Cent,Buy_it_now,No,$75.19,BIN,Free Shipping,Yes,Yes,8:39,Mar-2,2019,https://www.ebay.com/itm/2012-D-PCGS-MS67RD-Re...,35
4,Sold,1953 S PCGS MS67RD Red Lincoln Cent,Buy_it_now,No,$140.99,BIN,Free Shipping,Yes,Yes,8:39,Mar-2,2019,https://www.ebay.com/itm/1953-S-PCGS-MS67RD-Re...,35


# <center> Data Wrangling
### <center> (Creating More Useful Columns)

In [3]:
df1 = ebay_listings_df[['Result','Title','Format','Best_Offer_Accepted','Price','Bids','Shipping','Free_Returns','Top_Rated_Plus','Sale_Time','Sale_Date','Sale_Year','Title_Length','Link']]


# Refomrat Shipping cost

df1['Shipping']= df1['Shipping'].str.replace('Free Shipping', '0')
df1['Shipping']= df1['Shipping'].str.replace(' shipping', '')
df1['Shipping']= df1['Shipping'].str.replace('$', '')
df1['Shipping']= df1['Shipping'].str.replace('+', '')
df1['Shipping']= df1['Shipping'].astype(float)


# Create Price + Shipping colum 

df1['Price']= df1['Price'].str.replace('$', '')
df1['Price']= df1['Price'].str.replace(',', '')
df1['Price'] = df1['Price'].astype(float)
df1['Price_+_Shipping']= df1['Price']+ df1['Shipping']

# Convert Title to Upper Case

df1['Title'] = df1['Title'].str.upper()

# Create Grader column

df1.loc[df1['Title'].str.contains('PCGS'), 'Grader'] = 'PCGS'
df1.loc[df1['Title'].str.contains('NGC'), 'Grader'] = 'NGC'
df1.loc[df1['Title'].str.contains('ANACS'), 'Grader'] = 'ANACS'
df1.loc[df1['Title'].str.contains('ICG'), 'Grader'] = 'ICG'

# Drop Coins without listed Grader

df1['Length']= df1.Grader.str.len()
df1 = df1[df1.Length < 6 ]


# Create Date column

df1['Coin_Date'] = df1.Title.str.extract('(\d\d\d\d)')

# Crete CAC column

df1.loc[df1['Title'].str.contains(' CAC '), 'CAC'] = 'CAC'


# Create Mint column

df1.loc[df1['Title'].str.contains(' D '), 'Mint'] = 'D'
df1.loc[df1['Title'].str.contains('-D'), 'Mint'] = 'D'
df1.loc[df1['Title'].str.contains(' S '), 'Mint'] = 'S'
df1.loc[df1['Title'].str.contains('-S'), 'Mint'] = 'S'

# Replace NaN Values

df1['Mint'].fillna(' ', inplace=True)
df1['CAC'].fillna(' ', inplace=True)

# Refomat Bids 

df1['Bids']= df1['Bids'].str.replace(' bids', '')
df1['Bids']= df1['Bids'].str.replace(' bid', '')
df1['Bids']= df1['Bids'].str.replace('BIN', '-')

# Create Time Stamp Column

df1['Time_Stamp']= df1['Sale_Date']+ df1['Sale_Year']
df1['Time_Stamp'] = df1['Time_Stamp'].str.replace('201', '-201')
df1['Time_Stamp'] = pd.to_datetime(df1['Time_Stamp'])


# Create day of the week Column

df1['Sale_Day'] = df1['Time_Stamp'].dt.weekday_name

# Organize columns

fdf = df1[[
    'Coin_Date',
     'Mint',
    'Grader',
    'CAC',
    'Price_+_Shipping',
     'Price',
    'Shipping',
     'Bids',
    'Result',
    'Free_Returns',
    'Top_Rated_Plus',
     'Format',
  'Best_Offer_Accepted',
     'Title_Length',
     'Sale_Time',
    'Sale_Day',
     'Sale_Date',
    'Sale_Year',
    'Time_Stamp',
    'Link',
    'Title'
    
]]

# Save to CSV

fdf.to_csv(csv_title)

# View Data
fdf

,Coin_Date,Mint,Grader,CAC,Price_+_Shipping,Price,Shipping,Bids,Result,Free_Returns,...,Format,Best_Offer_Accepted,Title_Length,Sale_Time,Sale_Day,Sale_Date,Sale_Year,Time_Stamp,Link,Title
0,1954,S,PCGS,,122.95,120.00,2.95,-,Sold,Yes,...,Buy_it_now,No,76,9:41,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/1954-S-PCGS-MS67RD-1C...,1954-S PCGS MS67RD 1C LINCOLN WHEAT CENT PENNY...
1,2005,,PCGS,,25.52,19.99,5.53,-,Sold,No,...,Buy_it_now,No,80,8:43,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/PCGS-MS67-RD-2005-SAT...,PCGS MS67 RD 2005 SATIN FINISH RED LINCOLN MEM...
2,NaN,,PCGS,,667.00,639.50,27.50,0,Ended,No,...,Auction,No,79,8:40,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/LOW-RESERVE-Lot-of-25...,"LOW RESERVE! LOT OF 25 PCGS MS66RD, MS66+RD, ..."
3,2012,D,PCGS,,75.19,75.19,0.00,-,Sold,Yes,...,Buy_it_now,No,35,8:39,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/2012-D-PCGS-MS67RD-Re...,2012 D PCGS MS67RD RED LINCOLN CENT
4,1953,S,PCGS,,140.99,140.99,0.00,-,Sold,Yes,...,Buy_it_now,No,35,8:39,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/1953-S-PCGS-MS67RD-Re...,1953 S PCGS MS67RD RED LINCOLN CENT
5,1945,S,PCGS,,550.00,550.00,0.00,-,Ended,Yes,...,Buy_it_now,No,80,8:38,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/1945-S-Lincoln-Wheat-...,1945-S LINCOLN WHEAT CENT PCGS MS67+RD SUPERB ...
6,1942,S,NGC,,53.82,50.00,3.82,1,Ended,No,...,Auction,No,69,8:38,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/1942-S-Lincoln-Wheat-...,1942-S LINCOLN WHEAT CENT NGC MS 67 RD BRIGHT ...
7,1943,S,NGC,,675.00,675.00,0.00,-,Ended,Yes,...,Buy_it_now,No,78,8:37,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/1943-S-1C-NGC-MS67-SU...,1943-S 1C NGC MS67+ SUPERB GEM UNCIRCULATED HI...
8,1934,D,PCGS,,2475.00,2475.00,0.00,-,Ended,Yes,...,Buy_it_now,No,77,8:36,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/1934-D-1C-PCGS-MS67RD...,1934-D 1C PCGS MS67RD ONLY 4 GRADED HIGHER RED...
9,1940,S,NGC,,600.00,600.00,0.00,-,Ended,Yes,...,Buy_it_now,No,76,8:35,Saturday,Mar-2,2019,2019-03-02,https://www.ebay.com/itm/NGC-MS67-RD-1940-S-Li...,NGC MS67+RD 1940-S LINCOLN WHEAT CENT HIGH GRA...


### <center> Once you see the pandas data frame:
    
<center>Congratulations! Your CSV file has been saved in your choosen Location.